In [1]:
import numpy as np
from pyquery import PyQuery as pq
from matplotlib import pylab as plt

%matplotlib inline
%config InlineBackend.figure_format = "svg"
%config InlineBackend.close_figures = True

In [2]:
with open("data_txt/section1end.txt", 'r') as data_x_txt:
    data = data_x_txt.read()
    data = data.split()
    detrend_phi = np.array(data[1::2]).astype(np.float32)
    filtred_date = np.array(data[0::2]).astype(np.float32)
    print(filtred_date.shape, detrend_phi.shape)
with open("data_txt/section1end_nofil.txt", 'r') as data_x_txt:
    data = data_x_txt.read()
    data = data.split()
    phi = np.array(data[1::2]).astype(np.float32)
    print(phi.shape)

(13514,) (13514,)
(13514,)


In [3]:
max_peak = np.max(detrend_phi)
max_peak_i = np.where(detrend_phi == max_peak)[0][0]
max_peak_date = filtred_date[max_peak_i]

In [4]:
model_date = filtred_date[:max_peak_i]
model_phi = phi[:max_peak_i]

In [5]:
model_A = np.vstack([model_date, np.ones(len(model_date))]).T
model_a, model_b = np.linalg.lstsq(model_A, model_phi, rcond=None)[0]
print("a = {}, b = {}".format(model_a, model_b))
model_trend = model_a * model_date + model_b

a = -5.234587049937571, b = 10353.679664412059


In [6]:
detrend_model_phi = model_phi - model_trend

In [7]:
with open("data_txt/model_phi_1.txt", 'w') as datatxt:
    for i in range(len(model_date)) :
        print(model_date[i], '\t', detrend_model_phi[i], file=datatxt)

In [8]:
with open("data_txt/fmphi.txt", 'r') as data_txt:
    data = data_txt.read()
    data = data.split()
    filtred_model_phi = np.array(data[1::2]).astype(np.float32)
    print(filtred_model_phi.shape, model_date.shape)

(10477,) (10477,)


In [9]:
model_noise = detrend_model_phi - filtred_model_phi

In [10]:
N = 0.85
bayes = 1 - N
bayes

0.15000000000000002

In [11]:
def newmodel(repoint, old_date, old_phi, df1=0, df2=0, inte=0, lel=0):
    new_date1 = old_date[:repoint]
    new_date2 = old_date[repoint:]
    tmp_ = new_date2.size
    new_phi1 = old_phi[:repoint]
    if lel:
        tmp_ = lel

    new_phi2 = -1 *old_phi[repoint:] - (1 - N) * (old_date[:tmp_] -  new_date1[0]) * np.pi * 2
    new_phi2 += (new_phi1[-1] - new_phi2[0])

    if df1:
        plt.figure(figsize=(11.5, 15.5))
        plt.subplot(3,1,1)
        plt.grid(True)
        plt.ylabel("$\phi(t)$", fontsize=16)
        plt.xlabel("$t$")
        plt.xlim(old_date[0], old_date[-1])
        plt.plot(new_date1, new_phi1, color="black", label="Данные даблюдений")
        plt.plot(new_date2, new_phi2, color="red", label="Преобразованные данные")
        plt.legend()
        if df2:
            plt.subplot(3,1,2)
            plt.grid(True)
            plt.ylabel("$\phi(t)$", fontsize=16)
            plt.xlabel("$t$")
            plt.xlim(old_date[0], old_date[-1])
            plt.plot(old_date, old_phi, color="silver", label="$\phi(t)$")
            plt.plot(old_date[inte[0]: inte[1]], old_phi[inte[0]: inte[1]], color="lightpink", label="Интервал допустимых точек")
            plt.plot(old_date[repoint], old_phi[repoint],"o", c="blue", label="Выбранная точка для преобразования")
            plt.legend()

    
    return np.concatenate([new_date1, new_date2]), np.concatenate([new_phi1, new_phi2])

In [12]:
def find_max_delta(repoint, t, phi, length, h):
    new_t = t[:repoint]
    new_p = phi[:repoint]
    n = (new_t.size - length) // h
    a = []
    b = []
    res = []
    delta = []
    T = []
    PHI = []
    
    for i in range(n):
        t_ = np.array(new_t[i * h : length + i * h])
        T.append(t_)
        phi_ = np.array(new_p[i * h : length + i * h])
        PHI.append(phi_)

        A_ = np.vstack([T[i], np.ones(len(T[i]))]).T
        a_, b_ = np.linalg.lstsq(A_, PHI[i], rcond=None)[0]

        a.append(a_)
        b.append(b_)

        res_ = a_ * T[i] + b_

        res.append(res_)
        delta.append(a_)
        
    return np.min(delta)

In [6]:
def find_ppoint(repoint, t, phi, length, h, delta, df1=0, df2=0, old_date=None, old_phi=None,  inte = 0, df3=0):
    t_ex_ch = t[repoint]
    a = []
    b = []
    res = []
    
    n = (t.size - length) // h
    mmin = 5
    T = []
    PHI = []
    
    for i in range(n):
        t_ = np.array(t[i * h : length + i * h])
        T.append(t_)
        phi_ = np.array(phi[i * h : length + i * h])
        PHI.append(phi_)

        A_ = np.vstack([T[i], np.ones(len(T[i]))]).T
        a_, b_ = np.linalg.lstsq(A_, PHI[i], rcond=None)[0]

        a.append(a_)
        b.append(b_)

        res_ = a_ * T[i] + b_

        res.append(res_)
        if a[i] < -1 * np.abs(delta):
            t_ch = t[length + i * h] 
            ind_ch = length + i * h
            mmin = t_ch - t_ex_ch
            mmin_i = i
            
            break
          
    print("Точка перехода известная: %.4f"%t_ex_ch)
    print("Точка перехода alg: %.4f"%t[length + mmin_i * h])
    print("отклонение от точки перехода {}".format(mmin))
    print("a = {}".format(a[i]))
    print("\n")
    
    if df1:
        plt.figure(figsize=(11.5, 15.5))
        plt.subplot(3,1,1)
        plt.grid(True)
        plt.ylabel("$\phi(t)_{Модель}$")
        plt.xlabel("$t$")
        plt.xlim(t[0], t[-1])
        plt.plot(t, phi, label="$\phi(t)$", c="silver")
        plt.plot(t[repoint], phi[repoint],"o", c="r", label="Смена частотного режима(Фактическая)")
        plt.plot(t[length + mmin_i * h], phi[length + mmin_i * h],"o", c="lime", label="Смена частотного режима(Алгоритм)")
        plt.legend()

        if df2:
            plt.subplot(3,1,2)
            plt.grid(True)
            plt.ylabel("$\phi(t)_{real}$")
            plt.xlabel("$t$")
            plt.xlim(old_date[0], old_date[-1])
            plt.plot(old_date, old_phi, label="$\phi(t)$", c="silver")
            plt.plot(old_date[inte[0]: inte[1]], old_phi[inte[0]: inte[1]], c="lightpink", label="Допустимый интревал определения смены частотного режима")
            plt.plot(old_date[repoint], old_phi[repoint], "o", c="r", label="Смена частотного режима(Фактическая)")
            plt.plot(old_date[length + mmin_i * h], old_phi[length + mmin_i * h], "o", c="lime", label="Смена частотного режима(Алгоритм)")
            plt.legend()
    if df3:
        plt.figure(figsize=(11.5, 15.5))
        plt.subplot(3,1,2)
        plt.grid(True)
        plt.ylabel("$\phi(t)$")
        plt.xlabel("$t$")
        plt.xlim(old_date[0], old_date[-1])
        plt.plot(old_date, old_phi, color="black", label="$\phi(t)$")
        plt.plot(old_date[inte[0]: inte[1]], old_phi[inte[0]: inte[1]], c="lightpink", label="Допустимый интревал определения смены частотного режима")
        plt.plot(old_date[repoint], old_phi[repoint], "o", c="r", label="Смена частотного режима(Фактическая)")
        plt.plot(old_date[length + mmin_i * h], old_phi[length + mmin_i * h], "o", c="lime", label="Смена частотного режима(Алгоритм)")
        plt.legend()
    
    return t[length + mmin_i * h] - t_ex_ch, t[inte[1]] - t[length + mmin_i * h], t_ex_ch